In [28]:
import numpy as np
import random
import pandas as pd
from tqdm.notebook import tqdm
from neo4j import GraphDatabase

In [2]:
df = pd.read_csv("D:/Blackcoffer/Neo4j/ipl_2022_dataset.csv" )

In [475]:
df.head()

,Unnamed: 0,Player,Base Price,TYPE,COST IN ₹ (CR.),Cost IN $ (000),2021 Squad,Team
0,0,Rashid Khan,Draft Pick,BOWLER,15.0,1950.0,SRH,Gujarat Titans
1,1,Hardik Pandya,Draft Pick,ALL-ROUNDER,15.0,1950.0,MI,Gujarat Titans
2,2,Lockie Ferguson,2 Cr,BOWLER,10.0,1300.0,KKR,Gujarat Titans
3,3,Rahul Tewatia,40 Lakh,ALL-ROUNDER,9.0,1170.0,RR,Gujarat Titans
4,4,Shubman Gill,Draft Pick,BATTER,8.0,1040.0,KKR,Gujarat Titans


In [476]:
df = df[df["Team"]!="Unsold"]
df = df.drop(['Unnamed: 0'],axis=1)

df.columns = ['Player', 'Base Price', 'TYPE', 'cost',
       'Cost IN $ (000)', '2021 Squad', 'Team']

df['short_team']= df["Team"].apply(lambda x : "".join(e[0] for e in x.split()))
df['player_id'] = ["t"+str(i) for i in range(len(df))]

In [479]:
team_players={}
for t in df.Team.unique():
    target = df[df.Team==t]
    team_players[t] = target.player_id.values.tolist()

In [480]:
df["cost"] = df["cost"].apply(lambda x: x*10000000)

df["colleagues"] = df["Team"].apply(lambda x: team_players[x])
coll = []
for i, j in tqdm(enumerate(df.player_id.values)):
    values = [k for k in df.iloc[i]["colleagues"] if j != k] 
    coll.append(values)
    
df['colleagues'] = coll   

In [485]:
df.head()

,Player,Base Price,TYPE,cost,Cost IN $ (000),2021 Squad,Team,short_team,player_id,colleagues
0,Rashid Khan,Draft Pick,BOWLER,150000000.0,1950.0,SRH,Gujarat Titans,GT,t0,"[t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11,..."
1,Hardik Pandya,Draft Pick,ALL-ROUNDER,150000000.0,1950.0,MI,Gujarat Titans,GT,t1,"[t0, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11,..."
2,Lockie Ferguson,2 Cr,BOWLER,100000000.0,1300.0,KKR,Gujarat Titans,GT,t2,"[t0, t1, t3, t4, t5, t6, t7, t8, t9, t10, t11,..."
3,Rahul Tewatia,40 Lakh,ALL-ROUNDER,90000000.0,1170.0,RR,Gujarat Titans,GT,t3,"[t0, t1, t2, t4, t5, t6, t7, t8, t9, t10, t11,..."
4,Shubman Gill,Draft Pick,BATTER,80000000.0,1040.0,KKR,Gujarat Titans,GT,t4,"[t0, t1, t2, t3, t5, t6, t7, t8, t9, t10, t11,..."


In [29]:
class neo4j_python:
    def __init__(self, uri="bolt://localhost:7687", user = "neo4j", password = "password"):       
        self.data_base_connection = GraphDatabase.driver(uri = uri, auth=(user, password))
        self.session = self.data_base_connection.session()
        
    def execute_query(self,query, read= False):
        result =self.session.run(query)
        if read:
            return result
            
    def return_df(self,query):
        result = self.execute_query(query, read= True).data()
        return pd.DataFrame(result)

In [486]:
queries=[]

# create a node TEAM
# eg: (GT:TEAM{name: 'GT' })
for i in tqdm(df['short_team'].unique()):
    team = "("+str(i)+":TEAM{name: '"+str(i)+"' })"
    queries.append(team)
    
# create a node PLAYERS
# eg: (t5:PLAYER{name: 'Mohammad Shami' , type: 'BOWLER' })
for i in tqdm(range(len(df))):
    player = "("+str(df["player_id"].values[i].split()[0])+":PLAYER{name: '"+str(df["Player"].values[i])+"' , type: '"+str(df["TYPE"].values[i])+"' })"
    queries.append(player)

# create a relationship COLLEAGUE
# eg: (t0)-[:COLLEAGUE]->(t4)
for i in tqdm(range(len(df))):
    for j in df["colleagues"].values[i]:
        colleague = "(" +str(df["player_id"].values[i].split()[0])+ ")-[:COLLEAGUE]->" + "("+str(j)+")"
        queries.append(colleague)
        
# create a relationship SOLD_TO
# eg: (t154)-[:SOLD_TO{cost:15000000.0}]->(MI)       
for i in tqdm(range(len(df))):
    sold_to = "(" +str(df["player_id"].values[i].split()[0])+ ")-[:SOLD_TO{cost:"+str(df["cost"].values[i])+"}]->" + "("+str(df["short_team"].values[i])+")"
    queries.append(sold_to)

                            

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/237 [00:00<?, ?it/s]

  0%|          | 0/237 [00:00<?, ?it/s]

  0%|          | 0/237 [00:00<?, ?it/s]

In [488]:
final_query = ",".join(i for i in queries)
final_query = "CREATE "+ final_query

In [545]:
final_query

"CREATE (GT:TEAM{name: 'GT' }),(CSK:TEAM{name: 'CSK' }),(DC:TEAM{name: 'DC' }),(KKR:TEAM{name: 'KKR' }),(PK:TEAM{name: 'PK' }),(LSG:TEAM{name: 'LSG' }),(MI:TEAM{name: 'MI' }),(RCB:TEAM{name: 'RCB' }),(RR:TEAM{name: 'RR' }),(SH:TEAM{name: 'SH' }),(t0:PLAYER{name: 'Rashid Khan' , type: 'BOWLER' }),(t1:PLAYER{name: 'Hardik Pandya' , type: 'ALL-ROUNDER' }),(t2:PLAYER{name: 'Lockie Ferguson' , type: 'BOWLER' }),(t3:PLAYER{name: 'Rahul Tewatia' , type: 'ALL-ROUNDER' }),(t4:PLAYER{name: 'Shubman Gill' , type: 'BATTER' }),(t5:PLAYER{name: 'Mohammad Shami' , type: 'BOWLER' }),(t6:PLAYER{name: 'Yash Dayal' , type: 'BOWLER' }),(t7:PLAYER{name: 'David Miller' , type: 'BATTER' }),(t8:PLAYER{name: 'R. Sai Kishore' , type: 'BOWLER' }),(t9:PLAYER{name: 'Abhinav Sadarangani' , type: 'BATTER' }),(t10:PLAYER{name: 'Matthew Wade' , type: 'WICKETKEEPER' }),(t11:PLAYER{name: 'Alzarri Joseph' , type: 'BOWLER' }),(t12:PLAYER{name: 'Jason Roy' , type: 'BATTER' }),(t13:PLAYER{name: 'Wriddhiman Saha' , type: 'WI

Create Query

In [490]:
neo4j_session = neo4j_python()
neo4j_session.execute_query(final_query)

In [30]:
session = neo4j_python()

Query to return the name of Top 5 expensive players sold in IPL 2022 Auction with team Name and cost

In [8]:
session.return_df("MATCH (p:PLAYER) -[r:SOLD_TO]-> (t:TEAM)\
                   RETURN p.name AS name, r.cost AS Cost, t.name as team\
                   ORDER BY Cost DESC LIMIT 5")

,name,Cost,team
0,KL Rahul,170000000.0,LSG
1,Ravindra Jadeja,160000000.0,CSK
2,Rishabh Pant,160000000.0,DC
3,Rohit Sharma,160000000.0,MI
4,Ishan Kishan,152500000.0,MI


Query to return Players aquiered by Mumbai Indians in decending order by their buying cost

In [10]:
session.return_df("MATCH (p:PLAYER) -[r:SOLD_TO]-> (t:TEAM{name: 'MI' })\
                   RETURN p.name AS name, r.cost AS Cost\
                   ORDER BY Cost DESC LIMIT 5")

,name,Cost
0,Rohit Sharma,160000000.0
1,Ishan Kishan,152500000.0
2,Jasprit Bumrah,120000000.0
3,Tim David,82500000.0
4,Suryakumar Yadav,80000000.0


Query to return Colleagues of 'Ravindra Jadeja'

In [32]:
session.return_df("MATCH (p1:PLAYER{name: 'Ravindra Jadeja'}) -[r:COLLEAGUE]-> (p2:PLAYER)\
                   RETURN p2.name as Colleagues" )

,Colleagues
0,Chris Jordan
1,Mitchell Santner
2,C.Hari Nishaanth
3,Moeen Ali
4,Ambati Rayudu
5,Dwaine Pretorius
6,Dwayne Bravo
7,N. Jagadeesan
8,Rajvardhan Hangargekar
9,K.M. Asif


Query to return Colleagues of 'Rishabh Pant' who has sold for more than 5 cr rupees

In [35]:
session.return_df("MATCH (p1:PLAYER{name: 'Rishabh Pant'}) -[r:COLLEAGUE]-> (p2:PLAYER)\
                   MATCH (p3:PLAYER) -[rt:SOLD_TO]-> (t1:TEAM)\
                   WHERE p3.name IN p2.name AND rt.cost>50000000\
                   RETURN p3.name as name , rt.cost as cost" )

,name,cost
0,Axar Patel,90000000.0
1,Mitchell Marsh,65000000.0
2,Anrich Nortje,65000000.0
3,David Warner,62500000.0
4,Prithvi Shaw,75000000.0
5,Syed Khaleel Ahmed,52500000.0
6,Shardul Thakur,107500000.0


Query to return money spent by each team in IPL auction 2022

In [40]:
session.return_df("MATCH (t:TEAM)<-[rt:SOLD_TO]-(p:PLAYER)\
                       RETURN t.name as name , sum(rt.cost) as spent\
                       ORDER BY spent DESC" )

,name,spent
0,MI,899000000.0
1,SH,899000000.0
2,GT,896500000.0
3,LSG,894000000.0
4,RR,890500000.0
5,RCB,884500000.0
6,CSK,870500000.0
7,DC,864000000.0
8,PK,845500000.0
9,KKR,815500000.0
